In [1]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manyl

In [2]:
!pip install transformers

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [4]:
import torch

In [5]:
!pip install accelerate
!pip install -i https://pypi.org/simple/bitsandbytes

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00


In [6]:
MODEL_NAME = "IlyaGusev/saiga_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Альфачат, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

inputs = ["Почему трава зеленая?", "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч"]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

Почему трава зеленая?
Зеленый цвет является результатом фотосинтеза, процесса, который происходит в растениях для получения энергии из света. Когда свет попадает на хлоропласты (клетки, которые содержат зельевую пигментацию), они перерабатывают его в энергию, которая используется для синтеза сахаров и других органических веществ. Этот процесс приводит к образованию зеленого цвета.


Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч
Однажды в лесу была такая-то девочка по имени Таня. Она играла со своим лучшим другом - мячом. Мяч был ее верным другом и напарником во всех играх. Их игры были так же красивыми, как и природа вокруг них.

В один день они решили провести время вместе и отправились в путешествие. Вот что слу

Загрузка датасета

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Alfachat_dataset_-_Bank_CRM.csv')
data.head()

In [ ]:
data.drop('answer_start', axis=1, inplace=True)

In [ ]:
data.drop('title', axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
train = data[:len(data)-int(0.2*len(data))]
test = data[len(data)-int(0.2*len(data)):]

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
from json import loads, dumps

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
train.to_json('train.json', orient='records', lines=True)

In [ ]:
test.to_json('test.json', orient='records', lines=True)

In [ ]:
data = load_dataset(
    "json",
    data_files={
                'train' : 'train.json' ,
                'test' : 'test.json'
    }
)
data

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)

In [ ]:
CUTOFF_LEN = 3584


##{data_point['system']}</s><s>user
def generate_prompt(data_point):
    promt = f"""<s>system
{data_point['question']}</s><s>bot
{data_point['answer']}</s>"""
    #     print(promt)
    return promt


def tokenize (prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):

        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)



    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [ ]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["test"].map(generate_and_tokenize_prompt)
)

In [ ]:
import transformers

In [ ]:
from transformers import TrainingArguments

In [ ]:
BATCH_SIZE = 4
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 100
OUTPUT_DIR = "/tmp"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
#             warmup_steps=200,
            max_steps=TRAIN_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=10,
            save_steps=10,
            output_dir=OUTPUT_DIR,
            save_total_limit=10,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True, # Overwrite the content of the output dir
)

In [ ]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype= torch.float16,
#     bnb_4bit_use_double_quant=False,
# )


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype=torch.float16,
#     quantization_config=bnb_config
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
#      torch_dtype=torch.bfloat16,
    is_trainable = True,
#     device_map="auto"
).to(device)


model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)
trainer.train()
model.save_pretrained(OUTPUT_DIR)

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
# model = torch.compile(model)
trainer.train()

In [7]:
!pip3 install pytelegrambotapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 5.1 MB/s eta 0:00:00


In [8]:
from telebot import types

In [10]:
import telebot

In [46]:
bot = telebot.TeleBot(os['api_token]'])

@bot.message_handler(commands=['start'])
def start(message):

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("👋 Поздороваться")
    markup.add(btn1)
    bot.send_message(message.from_user.id, "👋 Привет! Я Альфачат, Ваш помощник по CRM системе!", reply_markup=markup)


@bot.callback_query_handler(func=lambda call: True)
def check_button(call):
   print(call.data)
   if call.data == "redirect":
        bot.answer_callback_query(call.id, "Ваш запрос зарегистрирован. С Вами свяжутся в ближайшие 30 минут.")
      #  await call.message.answer("Ваш запрос зарегистрирован. С Вами свяжутся в ближайшие 30 минут.")
   if call.data == "base_link":
        bot.answer_callback_query(call.id, 'Более подробную информацию можно получить в Базе Знаний по '  + '[ссылке](https://www.atlassian.com/ru/itsm/knowledge-management/what-is-a-knowledge-base)')
   if call.data == "close":
        button_1 = types.InlineKeyboardButton('1', callback_data='btn1')
        button_2 = types.InlineKeyboardButton('2', callback_data='btn2')
        button_3 = types.InlineKeyboardButton('3', callback_data='btn3')
        button_4 = types.InlineKeyboardButton('4', callback_data='btn4')
        button_5 = types.InlineKeyboardButton('5', callback_data='btn5')
        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(button_1, button_2, button_3, button_4, button_5)
        # bot.send_message(call.update.callback_query.from.id, "Пожалуйста, оцените работу бота", reply_markup=keyboard)
  #  await call.answer()

@bot.message_handler(content_types=['text'])
def get_text_messages(message):

    if message.text == '👋 Поздороваться':
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True) #создание новых кнопок
        btn1 = types.KeyboardButton('Как задать вопрос помощнику?')
        btn2 = types.KeyboardButton('Как пользоваться Базой знаний?')
        btn3 = types.KeyboardButton('Юридическая консультация')
        btn4 = types.KeyboardButton('Оценить работу бота')
        markup.add(btn1, btn2, btn3, btn4)
        bot.send_message(message.from_user.id, '❓ Задайте интересующий вас вопрос или выберите опцию ниже', reply_markup=markup) #ответ бота


    elif message.text == 'Как задать вопрос помощнику?':
        bot.send_message(message.from_user.id, 'Если вы хотите задать вопрос, напишите его в чате. Если я не смогу Вам помочь, то запрос автоматически перенаправится на горячую линию и с Вами свяжутся, Вам не придется формулировать запрос заново.', parse_mode='Markdown')

    elif message.text == 'Как пользоваться Базой знаний?':
        bot.send_message(message.from_user.id, 'Воспользоваться базой знаний можно по'  + '[ссылке](https://www.atlassian.com/ru/itsm/knowledge-management/what-is-a-knowledge-base)', parse_mode='Markdown')

    elif message.text == 'Юридическая консультация':
        bot.send_message(message.from_user.id, 'Воспользуйтесь сервисом Констультант по ' + '[ссылке](https://www.consultant.ru/)', parse_mode='Markdown')

    elif message.text == 'Перенаправить вопрос на горячую линию ❌':
        bot.send_message(message.from_user.id, 'Ваш запрос зарегистрирован. С Вами свяжутся в ближайшие 30 минут.', parse_mode='Markdown')

    elif message.text == 'Получить ссылку на Базу Знаний с вопросом 💻':
        bot.send_message(message.from_user.id, 'Более подробную информацию можно получить в Базе Знаний по '  + '[ссылке](https://www.atlassian.com/ru/itsm/knowledge-management/what-is-a-knowledge-base)', parse_mode='Markdown')

    elif message.text == 'Оценить работу бота':
        button_1 = types.InlineKeyboardButton('1', callback_data='btn1')
        button_2 = types.InlineKeyboardButton('2', callback_data='btn2')
        button_3 = types.InlineKeyboardButton('3', callback_data='btn3')
        button_4 = types.InlineKeyboardButton('4', callback_data='btn4')
        button_5 = types.InlineKeyboardButton('5', callback_data='btn5')
        keyboard = types.InlineKeyboardMarkup()
        keyboard.add(button_1, button_2, button_3, button_4, button_5)
        bot.send_message(message.from_user.id, 'Пожалуйста, оцените работу бота', reply_markup=keyboard)

    else:
      inputs = [message.text]
      for inp in inputs:
          markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
          conversation = Conversation()
          conversation.add_user_message(inp)
          prompt = conversation.get_prompt(tokenizer)

          output = generate(model, tokenizer, prompt, generation_config)

          button_foo = types.InlineKeyboardButton('Перенаправить вопрос на горячую линию ❌', callback_data='redirect')
          button_bar = types.InlineKeyboardButton('Получить ссылку на Базу Знаний с вопросом 💻', callback_data='base_link')
          button_knowledge_base = types.InlineKeyboardButton('Ответ получен ✅', callback_data='close')

          keyboard = types.InlineKeyboardMarkup()
          keyboard.add(button_foo)
          keyboard.add(button_bar)
          keyboard.add(button_knowledge_base)


          # bot.send_message(message.from_user.id, output, parse_mode='Markdown')
          bot.send_message(message.from_user.id, output, reply_markup=keyboard)


bot.polling(none_stop=True, interval=0) #обязательная для работы бота часть

redirect
redirect
redirect
redirect
